<center> <img src = https://upload.wikimedia.org/wikipedia/commons/7/79/HeadHunter_logo.png alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import numpy as np
import psycopg2

In [2]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = '*******'
HOST = '_._._._'
PORT = 0000

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
query_3_1 = f'''select count(name)
                from vacancies  
'''

In [5]:
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_8956\3851344990.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [1]:
query_3_2 = f'''select count (name)
                from employers                
'''

In [6]:
df = pd.read_sql_query(query_3_2, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_10184\3588252056.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_2, connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [40]:
query_3_3 = f'''select count (distinct name)
                from areas
'''

In [42]:
df = pd.read_sql_query(query_3_3, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_2564\2018224393.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_3, connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [14]:
query_3_4 = f'''select count (distinct (name))
                from industries
'''

In [15]:
df = pd.read_sql_query(query_3_4, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_17816\2172479966.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_4, connection)


,count
0,294


***

# Выводы по предварительному анализу данных.

В среднем на одну сферу деятельности приходится 167 вакансий.

Соотношение работодателей к вакансиям примерно 0.45

В среднем на регион 36 вакансий.

Можно с уверенность сказать что:
- количество регионов (таблица areas) имеет отношение не только к России(так как 1362, больше чем городов РФ)
- количество сфер деятельности в базе (таблица industries) слишком велико(294), можно их сгруппировать.
- количество вакансий в нашей базе (таблица vacancies) в соотношении к количеству работодателей (таблица employers) примерно = 1/2,
что говорит о хороших шансе найти работу.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [128]:
query_4_1 = f'''select a.name Region, count(v.name)
from vacancies v join areas a on v.area_id = a.id
group by a.name
order by count(v.name) desc
limit 20
'''

In [129]:
df = pd.read_sql_query(query_4_1, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_23992\347208320.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_1, connection)


,region,count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [47]:
query_4_2 = f'''select count(name)
from vacancies
where salary_from is not null
or salary_to is not null
'''

In [48]:
df = pd.read_sql_query(query_4_2, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_2564\2168503709.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_2, connection)


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [136]:
query_4_3 = f'''select round(avg(salary_from)) "Нижняя граница", round(avg(salary_to)) "Верхняя граница"
from vacancies
'''

In [137]:
df = pd.read_sql_query(query_4_3, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_23992\2222372203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_3, connection)


,Нижняя граница,Верхняя граница
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [15]:
query_4_4 = f'''select schedule "Тип рабочего графика", employment "Тип трудоустройства", count(schedule)
from vacancies
group by schedule, employment
order by count(schedule) desc
'''

In [16]:
df = pd.read_sql_query(query_4_4, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_10184\2981602351.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_4, connection)


,Тип рабочего графика,Тип трудоустройства,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [25]:
query_4_5 = f'''(select experience "Опыт", count(experience) "Количество"
from vacancies
group by experience)
order by 2
'''

In [26]:
df = pd.read_sql_query(query_4_5, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_10184\2601166264.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_5, connection)


,Опыт,Количество
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


# Выводы по детальному анализу данных.

- Количество вакансий в каждом регионе прямо зависит от его численности жителей.
- Примерно у половины вакансий не заполнено хотя бы одно из двух полей с зарплатой. Так же вилка зарплат отличается примерно на 30%, что указывает на определенную категорию опыта вакансий, где указанно поле ЗП.
- Количество вакансий для каждого сочетания типа рабочего графика и типа трудоустройства для подовляющего большинства (около 75%) будет Полный день/Полная занятость, что говорит о поиске постоянной и основной работы.
- Больше половины вакансий требуется опыт работы от 1 года до 3 лет.

***

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [27]:
query_5_1 = f'''(select count(v.employer_id), e.name
from employers e join vacancies v on e.id = v.employer_id
group by e.name
order by 1 desc
limit 1)
union
(select count(v.employer_id), e.name
from employers e join vacancies v on e.id = v.employer_id
group by e.name
order by 1 desc
offset 4 limit 1)
order by 1 desc
'''

In [28]:
df = pd.read_sql_query(query_5_1, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_10184\1792814825.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_1, connection)


,count,name
0,1933,Яндекс
1,331,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [57]:
query_5_2 = f'''select a.name, count(e.id) "Работодатель", count(v.id) "Вакансии"
from areas a join employers e on a.id = e.area left join vacancies v on a.id = v.area_id
where v.id is null
group by 1
order by 3, 2 desc
'''

In [58]:
df = pd.read_sql_query(query_5_2, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_10184\3341481329.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_2, connection)


,name,Работодатель,Вакансии
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
181,Миллерово,1,0
182,"Михайловка (Уфимский район, Башкортостан)",1,0
183,Можайск,1,0
184,Морозовск\n,1,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [13]:
query_5_3 = f'''select e.name "Компания", count(distinct v.area_id) "Регионов присутсвия"
from vacancies v full join employers e on v.employer_id = e.id
group by 1
order by 2 desc
'''

In [14]:
df = pd.read_sql_query(query_5_3, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_10152\2398953733.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_3, connection)


,Компания,Регионов присутсвия
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [11]:
query_5_4 = f'''select count(id)
from employers left join employers_industries on id = employer_id
where industry_id is null
'''

In [12]:
df = pd.read_sql_query(query_5_4, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_10152\3114411492.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_4, connection)


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [8]:
query_5_5 = f'''select name "Компания", count(industry_id) "Количество сфер"
from employers left join employers_industries on id = employer_id
group by 1
having count(industry_id) = 4
order by 1
offset 2 limit 1
'''

In [9]:
df = pd.read_sql_query(query_5_5, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_17244\1650419958.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_5, connection)


,Компания,Количество сфер
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [3]:
query_5_6 = f'''select name "Сфера деятельности", count(distinct employer_id)
from industries join employers_industries on id = industry_id
where name like 'Разработка программного обеспечения'
group by 1
'''

In [6]:
df = pd.read_sql_query(query_5_6, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_17684\1169039133.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_6, connection)


,Сфера деятельности,count
0,Разработка программного обеспечения,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [84]:
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8#%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D0%BE%D0%B2'

cities= pd.read_html(url)[1].Город.replace('Воронеж[a]', 'Воронеж')
print(cities)

0              Москва
1     Санкт-Петербург
2         Новосибирск
3        Екатеринбург
4              Казань
5     Нижний Новгород
6          Красноярск
7           Челябинск
8              Самара
9                 Уфа
10     Ростов-на-Дону
11               Омск
12          Краснодар
13            Воронеж
14              Пермь
15          Волгоград
Name: Город, dtype: object


In [91]:
query_5_7 = f'''(select a.name "Города миллионики", count(e.name) cnt
from vacancies v full join areas a on v.area_id = a.id full join employers e on v.employer_id = e.id
where a.name in ('Москва', 'Санкт-Петербург','Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 
'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград') and e.name = 'Яндекс'
group by a.name)
union
(select 'total' "Города миллионики", count(e.name) cnt
from vacancies v full join areas a on v.area_id = a.id full join employers e on v.employer_id = e.id
where a.name in ('Москва', 'Санкт-Петербург','Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 
'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград') and e.name = 'Яндекс')
order by cnt
'''

In [92]:
df = pd.read_sql_query(query_5_7, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_10184\3129374555.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_7, connection)


,Города миллионики,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

# выводы по анализу работодателей

In [150]:
empl_info = f'''select count(v.employer_id), e.name
from employers e join vacancies v on e.id = v.employer_id
group by e.name
order by 1 desc
limit 10
'''
df = pd.read_sql_query(empl_info, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_23992\2609541901.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(empl_info, connection)


,count,name
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть
5,327,ИК СИБИНТЕК
6,292,МТС
7,247,DataArt
8,204,Совкомбанк Технологии
9,176,Первый Бит


Компания 'Яндекс' является лидером по количеству вакансий(отрыв более 5-ти раз от следующей компании), а так же по представлению в регионах.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [77]:
query_6_1 = f'''select count(name)
from vacancies
where lower(name) like '%data%'
or lower(name) like '%данн%'
'''

In [78]:
df = pd.read_sql_query(query_6_1, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_23992\485020324.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_1, connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [67]:

query_6_2 = f'''select count(name)
from vacancies
where (lower(name) like '%data scientist%'
or lower(name) like '%data science%'
or lower(name) like '%иследователь данных%'
or (name like '%ML%'
and name not like '%HTML%')
or lower(name) like '%machine learning%'
or lower(name) like '%машинн%обучен%%')
and (lower(name) like '%junior%'
or experience = 'Нет опыта'
or employment = 'Стажировка')
'''

In [26]:
ds_key = '''(lower(name) like '%data scientist%'
or lower(name) like '%data science%'
or lower(name) like '%иследователь данных%'
or (name like '%ML%'
and name not like '%HTML%')
or lower(name) like '%machine learning%'
or lower(name) like '%машинн%обучен%%')
'''

In [68]:
df = pd.read_sql_query(query_6_2, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_23992\3641650667.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [105]:
query_6_3 = f'''select count(*)
from vacancies
where {ds_key} 
and (upper(key_skills) like '%SQL%'
or lower(key_skills) like '%postgres%')
'''

In [106]:
df = pd.read_sql_query(query_6_3, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_23992\1159587760.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_3, connection)


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [107]:
query_6_4 = f'''select count(*)
from vacancies
where {ds_key} 
and lower(key_skills) like '%python%'
'''

In [108]:
df = pd.read_sql_query(query_6_4, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_23992\972386973.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4, connection)


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [109]:
query_6_5 = f'''select round(avg(1+length(key_skills)-length(replace(key_skills, chr(9), ''))), 2)
from vacancies
where {ds_key} 
and key_skills is not null
'''

In [110]:
df = pd.read_sql_query(query_6_5, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_23992\390248827.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_5, connection)


,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [119]:
query_6_6 = f'''select experience, round(avg((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from)) /2 )) 
from vacancies
where {ds_key} 
and (salary_to is not null
or salary_from is not null)
group by experience
'''

In [120]:
df = pd.read_sql_query(query_6_6, connection)
df

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_23992\3561045109.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6, connection)


,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

# выводы по предметному анализу

Для начинающего специалиста DS, есть подходящее количество вакансий. Основные навыки проходятся в курсе обучения. Зарплаты даже начинающего специалиста выше средней зп по стране. Продалжаем учится и устраиваемся на работу )

# Общий вывод по проекту

В 2019 году был очень большой бум на цифровые профессии, а именно:

- У больших федеральных компаний пул вакансий был не только в больших городах, но почти в кажом регионе, что говорит о бурном росте.
- Цифровые профессии нужны так же и небольшим компаниям, что говорит о оргомном присутсвии на рынке труда и их в огромном количестве.
- Если взять данные, то скорее всего тренд будет сохрянятся.(что и подтвердят 2020,2021,2022 года)

Если взять профессию Data Science, то объем от всех вакансий состовляет примерно 3.5%. Из этого можно сделать вывод:

- Что это молодая профессия, которая только будет набирать обороты.
- Это универсальная профессия, которая требует большого количества знаний(в среднем 6 навыков)
- Без опыта зайти в эту профессию будет очень сложно, так как вакансий совсем небольшое колличество(51)
- С другой стороны, заработная плата без опыта в 2 раза меньше чем у опыта от 1 года и в 3.5 раза от 3 лет. Есть смысл вкладываться в молодых специалистов.


Для более глубокого анализа, возьмем данные по городу Санкт-Петербург, так-как наша небольшая компания находиться там.

In [42]:
total = f'''select count(a.name), v.name
from vacancies v join areas a on v.area_id = a.id
where a.name like 'Санкт-Петербург'
group by 2
order by 1 desc
limit 20
'''
spb_1 = pd.read_sql_query(total, connection) # информация по вакансиям, города спб
spb_1


C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_2556\1302860327.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  spb_1 = pd.read_sql_query(total, connection) # информация по вакансиям, города спб


,count,name
0,68,Системный администратор
1,46,Бизнес-аналитик
2,35,Системный аналитик
3,32,Специалист технической поддержки
4,30,Аналитик
5,26,Frontend-разработчик
6,16,Программист 1С
7,14,Инженер технической поддержки
8,14,Менеджер проектов
9,13,Java-разработчик


In [30]:
jun = f'''select count(a.name), v.name
from vacancies v join areas a on v.area_id = a.id
where (lower(v.name) like '%junior%'
or v.experience = 'Нет опыта'
or v.employment = 'Стажировка')
and a.name like 'Санкт-Петербург'
group by 2
order by 1 desc
limit 30
'''
spb_2 = pd.read_sql_query(jun, connection) # информация по вакансиям, города спб без опыта
spb_2

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_2556\2740066079.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  spb_2 = pd.read_sql_query(ds_jun, connection) # информация по вакансиям, города спб


,count,name
0,10,Специалист технической поддержки
1,7,Системный администратор
2,6,Специалист службы поддержки пользователей
3,3,Помощник системного администратора
4,3,Младший аналитик
5,3,Аналитик
6,3,Специалист по информационной безопасности
7,2,Тестировщик / Junior QA
8,2,Программист-стажер 1С
9,2,Специалист по технической поддержке


In [43]:
ds = f'''select count(a.name), v.name
from vacancies v join areas a on v.area_id = a.id
where (lower(v.name) like '%data%'
or lower(v.name) like '%данн%')
and a.name like 'Санкт-Петербург'
group by 2
order by 1 desc
limit 10
'''
spb_3 = pd.read_sql_query(ds, connection) # # информация по вакансиям, города спб по специальности ds
spb_3

C:\Users\Psa1kl\AppData\Local\Temp\ipykernel_2556\1699587184.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  spb_3 = pd.read_sql_query(ds, connection) # информация по вакансиям, города спб


,count,name
0,11,Data Analyst
1,8,Аналитик данных
2,7,Data Engineer
3,7,Data Scientist
4,2,Data analyst
5,2,Senior Data Scientist
6,2,Team Lead Data Science
7,2,Junior Data Scientist
8,2,Data scientist
9,2,ML Engineer/ Data Scientist (WAF)


Вывод по анализу вакансий в городе Санкт-Петербург:
- вакансий Data Scientist очень мало, для джуна всего 2
- можно начать работу без опыта по профессиям аналитика и инженерия данных.